# Load SNP orders

In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "SNP"

# Set the root
from from_root import from_root # type: ignore
ROOT = from_root()

import pandas as pd # type: ignore
from loguru import logger # type: ignore

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

from pathlib import Path
import sys

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util # type: ignore

util.startLoop()

logger.add(sink=ROOT / "log" / "snp.log", mode="w")

1

## Helper functions

In [2]:
def yes_or_no(question: str, default="n") -> bool:
    """Asks a yes or no question with a default answer.

    Args:
      question: The question to ask.
      default: The default answer if the user presses Enter.

    Returns:
      True if the user answered yes, False otherwise.
    """

    while True:
        answer = input(question + " (y/n): ").lower().strip()
        if not answer:
            return default == "y"
        if answer in ("y", "yes"):
            return True
        elif answer in ("n", "no"):
            return False
        else:
            print("Please answer yes or no.")

# Get orders to be placed

In [3]:
# Check age of pickles

from src.snp import get_pickle, how_many_days_old
nakeds_path = ROOT / 'data' / 'snp_nakeds.pkl'
df_opts = get_pickle(nakeds_path)

txt = f'snp_nakeds.pkl is {how_many_days_old(nakeds_path): 0.2f} days old. Want to load??'
ans = yes_or_no(txt)

if ans:
    print('\n\n')
    cols = 'ib_symbol,strike,safe_strike,right,dte,bsPrice,price,xPrice,margin,rom'.split(',')
    print(df_opts[cols].head())
else:
    print('Bye!!!')




  ib_symbol  strike  safe_strike right   dte  bsPrice  price  xPrice   margin  \
0      GDDY   150.0          154     P  3.84     0.04   1.45    1.45   6589.0   
1       CZR    42.0           42     P  3.84     0.07   0.24    0.24   1818.4   
2      INTC    21.0           21     P  3.84     0.06   0.32    0.32    950.0   
3       CAH   106.0          107     P  3.84     0.05   1.02    1.02   4534.0   
4        CI   297.5          301     P  3.84     0.23   1.95    1.95  12826.6   

    rom  
0  2.09  
1  1.26  
2  3.20  
3  2.14  
4  1.45  


# Remove open orders from orders to plant

In [4]:
from src.snp import get_ib, get_open_orders, quick_pf

# check open orders
with get_ib('SNP') as ib:
    dfo = get_open_orders(ib)
    dfp = quick_pf(ib)

if not dfo.empty:
    remove_opens = set(dfo.symbol.to_list())
else:
    remove_opens = set()

# make a list of symbols to be removed from df_opts
if not dfp.empty:
    remove_positions = set(dfp.symbol.to_list())
else:
    remove_positions = set()

remove_ib_syms = remove_opens | remove_positions

# get the target options to plant
dft = df_opts[~df_opts.ib_symbol.isin(remove_ib_syms)].reset_index(drop=True)

print(f'{len(dft)} target orders found\n\n')

dft.head()

3525 target orders found




,ib_symbol,conId,expiry,strike,right,contract,price,undId,secType,undPrice,und_iv,und_hv,dte,iv,sdev,safe_strike,intrinsic,bsPrice,margin,comm,lot,xPrice,rom
0,GDDY,728220917,2024-11-01 10:00:00+00:00,150.0,P,"Option(conId=728220917, symbol='GDDY', lastTra...",1.45,188986978,STK,161.10,0.34,0.17,3.84,0.34,5.56,154,154.0,0.04,6589.0,0.65,1,1.45,2.09
1,CZR,728212663,2024-11-01 10:00:00+00:00,42.0,P,"Option(conId=728212663, symbol='CZR', lastTrad...",0.24,435774408,STK,44.86,0.45,0.37,3.84,0.45,2.08,42,42.0,0.07,1818.4,0.65,1,0.24,1.26
2,INTC,728229226,2024-11-01 10:00:00+00:00,21.0,P,"Option(conId=728229226, symbol='INTC', lastTra...",0.32,270639,STK,22.95,0.66,0.52,3.84,0.66,1.55,21,21.0,0.06,950.0,0.65,1,0.32,3.20
3,CAH,728208701,2024-11-01 10:00:00+00:00,106.0,P,"Option(conId=728208701, symbol='CAH', lastTrad...",1.02,5407,STK,110.80,0.26,0.15,3.84,0.26,2.92,107,107.0,0.05,4534.0,0.65,1,1.02,2.14
4,CI,737623517,2024-11-01 10:00:00+00:00,297.5,P,"Option(conId=737623517, symbol='CI', lastTrade...",1.95,346403540,STK,315.79,0.36,0.20,3.84,0.36,11.52,301,301.0,0.23,12826.6,0.65,1,1.95,1.45


# PREPARE THE ORDERS

In [5]:
from src.snp import load_config

config = load_config(MARKET)
MARGINPERORDER = config.get('MARGINPERORDER')
MARGINPERORDER

15000

In [6]:
dfs = dft.groupby('ib_symbol').apply(lambda x: x.sort_values('rom', ascending=False), include_groups=False).reset_index(level='ib_symbol')
df_target = dfs.groupby('ib_symbol', group_keys=True).apply(lambda x: (x[x['margin'].cumsum() < MARGINPERORDER]), include_groups=False).reset_index(level='ib_symbol')
df_target.reset_index(drop=True)

,ib_symbol,conId,expiry,strike,right,contract,price,undId,secType,undPrice,und_iv,und_hv,dte,iv,sdev,safe_strike,intrinsic,bsPrice,margin,comm,lot,xPrice,rom
0,AAPL,735049399,2024-11-01 10:00:00+00:00,247.5,P,"Option(conId=735049399, symbol='AAPL', lastTra...",15.00,265598,STK,232.79,0.26,0.23,3.84,0.26,6.29,225,225.0,14.59,9340.6,0.65,1,37.50,38.20
1,ABBV,728184998,2024-11-01 10:00:00+00:00,250.0,P,"Option(conId=728184998, symbol='ABBV', lastTra...",62.15,118089500,STK,189.18,0.29,0.16,3.84,0.29,5.68,182,182.0,60.68,8715.0,0.65,1,130.15,142.09
2,ABNB,731836041,2024-11-08 10:00:00+00:00,150.0,P,"Option(conId=731836041, symbol='ABNB', lastTra...",16.69,459530964,STK,135.94,0.48,0.30,10.84,0.48,11.28,122,122.0,14.53,5700.6,0.65,1,44.69,26.41
3,ABNB,733175262,2024-11-01 10:00:00+00:00,136.0,P,"Option(conId=733175262, symbol='ABNB', lastTra...",1.90,459530964,STK,135.94,0.48,0.30,3.84,0.48,6.71,127,127.0,2.67,5621.6,0.65,1,11.67,19.75
4,ABT,736395484,2024-11-01 10:00:00+00:00,134.0,P,"Option(conId=736395484, symbol='ABT', lastTrad...",19.78,4065,STK,113.94,0.21,0.17,3.84,0.21,2.43,111,111.0,19.98,4529.6,0.65,1,42.98,90.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,WYNN,734607312,2024-11-01 10:00:00+00:00,114.0,P,"Option(conId=734607312, symbol='WYNN', lastTra...",16.01,16454492,STK,99.18,0.34,0.39,3.84,0.62,6.30,91,91.0,14.79,4086.2,0.65,1,39.01,90.84
787,WYNN,733323154,2024-11-01 10:00:00+00:00,113.0,P,"Option(conId=733323154, symbol='WYNN', lastTra...",15.01,16454492,STK,99.18,0.34,0.39,3.84,0.62,6.30,91,91.0,13.81,4086.2,0.65,1,37.01,86.18
788,XOM,734598917,2024-11-01 10:00:00+00:00,134.0,P,"Option(conId=734598917, symbol='XOM', lastTrad...",14.51,13977,STK,117.65,0.24,0.23,3.84,0.24,2.95,114,114.0,16.27,4522.0,0.65,1,36.27,76.32
789,XOM,722321356,2024-11-15 10:00:00+00:00,145.0,P,"Option(conId=722321356, symbol='XOM', lastTrad...",26.12,13977,STK,117.65,0.24,0.23,17.84,0.24,6.36,110,110.0,26.96,4583.0,0.65,1,61.96,27.67


In [7]:
from ib_async import LimitOrder
from snp import get_prec

def make_ib_orders(df: pd.DataFrame) -> tuple:
    """Make (contract, order) tuples"""

    contracts = df.contract.to_list()
    orders = [
        LimitOrder(action="SELL", totalQuantity=abs(int(q)), lmtPrice=get_prec(p, 0.01))
        for q, p in zip(df.lot, df.xPrice)
    ]

    cos = tuple((c, o) for c, o in zip(contracts, orders))

    return cos

In [8]:
# Make contract orders of the target
cos = make_ib_orders(df_target)

In [9]:
cos

((Option(conId=735049399, symbol='AAPL', lastTradeDateOrContractMonth='20241101', strike=247.5, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='AAPL  241101P00247500', tradingClass='AAPL'),
  LimitOrder(action='SELL', totalQuantity=1, lmtPrice=37.5)),
 (Option(conId=728184998, symbol='ABBV', lastTradeDateOrContractMonth='20241101', strike=250.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='ABBV  241101P00250000', tradingClass='ABBV'),
  LimitOrder(action='SELL', totalQuantity=1, lmtPrice=130.15)),
 (Option(conId=731836041, symbol='ABNB', lastTradeDateOrContractMonth='20241108', strike=150.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='ABNB  241108P00150000', tradingClass='ABNB'),
  LimitOrder(action='SELL', totalQuantity=1, lmtPrice=44.69)),
 (Option(conId=733175262, symbol='ABNB', lastTradeDateOrContractMonth='20241101', strike=136.0, right='P', multiplier='100', exchange='SMART', currency='USD'

# PLACE THE ORDERS

In [10]:
# PLACES THE ORDERS
# =================

from datetime import datetime
from src.snp import pickle_me, place_orders


with get_ib('SNP') as ib:
    ordered = place_orders(ib=ib, cos=cos)
    
filename = f"{datetime.now().strftime('%Y%m%d_%I_%M_%p')}_snp_naked_orders.pkl"
pickle_me(ordered, str(ROOT / "data" / "history" / str(filename)))

100%|██████████| 32/32 [00:25<00:00,  1.24it/s]
